# 8. 일급함수, 익명함수, callable, 매개변수, signature, partial

### 일급함수 (일급 객체) - 파이썬 함수의 특징 
1. 런타임 초기화 가능 
2. 변수에 할당 가능 
3. 함수 인수에 전달 가능 (ex) sorted(keys=len) - 함수 안에 함수 
4. 함수 결과로 반환 가능 (ex) return function 

## (1) 팩토리얼 함수 구현 예제

In [1]:
def factorial(n):
    print('run')
    """Factorial Function -> n: int"""
    if n == 1: # n < 2
        return 1 
    return n* factorial(n-1)

# 함수와 비교를 위해 생성 
class A:
    pass 

In [2]:
print(factorial(5))
print(factorial.__doc__)
print(type(factorial), type(A))
print(set(sorted(dir(factorial))) - set(sorted(dir(dir(A))))) # 클래스와 다르게 함수가 갖고있는 메소드들
print(factorial.__name__) # 함수 이름
print(factorial.__code__) # 함수 코드 위치

run
run
run
run
run
120
None
<class 'function'> <class 'type'>
{'__globals__', '__get__', '__qualname__', '__closure__', '__defaults__', '__name__', '__kwdefaults__', '__code__', '__call__', '__module__', '__dict__', '__annotations__'}
factorial
<code object factorial at 0x000001D05EED9930, file "<ipython-input-1-a438f89c7b43>", line 1>


- __변수 할당/ 함수 인자 전달__

In [5]:
val_func = factorial 
print(val_func)
print(val_func(5))
print()
print(map(val_func, range(1, 6)))
print()
print(list(map(val_func, range(1, 6))))

<function factorial at 0x000001D05EF3A0D0>
run
run
run
run
run
120


run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
[1, 2, 6, 24, 120]


- 함수 인수 전달 및 함수로 결과 반환 -> __고위함수 (Higher-order Fuction)__

In [7]:
print(list(map(val_func, filter(lambda x: x%2, range(1, 6))))) # 함수 안에 함수 전달 
print()
print([val_func(i) for i in range(1, 6) if i%2])

run
run
run
run
run
run
run
run
run
[1, 6, 120]

run
run
run
run
run
run
run
run
run
[1, 6, 120]


## (2) reduce
- lambda, map(), reduce(), filter() 참고 <br> 
https://wikidocs.net/64

In [9]:
from functools import reduce 
import functools
from operator import add 

print(reduce(add, range(1, 11))) # 누적 
print(sum(range(1, 11)))

55
55


- __익명함수 (lambda) 와 reduce__
- __익명함수__
 1. 가급적 주석 사용 (가독성이 떨어지기 때문)
 2. 가급적 함수 사용
 3. 일반 함수 형태로 리팩토링 권장
 

In [10]:
print(reduce(lambda x, t: x + t, range(1, 11)))
# 1 + 2 ---> 3 + 3 ---> 6 + 4 ---> 10 + 5 ---> ...

55


## (3) Callable: 호출 연산자
: 메소드 형태로 호출 가능한지 확인

- **로또 추첨 클래스 예제**

In [19]:
# 클래스 선언 
import random 

class LottoGame:
    def __init__(self): 
        self._balls = [n for n in range(1, 46)]
    
    def pick(self): 
        random.shuffle(self._balls)
        return sorted([random.choice(self._balls) for _ in range(6)])

In [20]:
# 객체 생성
game = LottoGame()

In [22]:
# 로또 추첨 실행
print(game.pick())

[6, 11, 14, 24, 32, 35]


- callable 사용

In [23]:
print(callable(str), callable(list), callable(factorial), callable(3.14), callable(game))

True True True False False


- __call__ method 사용 (매우 중요한 개념)

In [26]:
# 클래스 선언 
import random 

class LottoGame:
    def __init__(self): 
        self._balls = [n for n in range(1, 46)]
    
    def pick(self): 
        random.shuffle(self._balls)
        return sorted([random.choice(self._balls) for _ in range(6)])
    
    def __call__(self): 
        return self.pick()

In [28]:
game = LottoGame()

In [29]:
print(game())
print(callable(game))

[14, 16, 39, 39, 41, 41]
True


## (4) 매개변수 입력 (*args, **kwargs)

In [31]:
def args_test(name, *contents, point=None, **attrs): 
    return '<args_test> -> (name: {}) (contents: {}) (point: {}) (attrs: {})'.format(name, contents, point, attrs)

In [32]:
print(args_test('test1'))

<args_test> -> (name: test1) (contents: ()) (point: None) (attrs: {})


In [34]:
print(args_test('test1', 'test2'))

<args_test> -> (name: test1) (contents: ('test2',)) (point: None) (attrs: {})


In [36]:
print(args_test('test1', 'test2', 'test3', id='admin'))

<args_test> -> (name: test1) (contents: ('test2', 'test3')) (point: None) (attrs: {'id': 'admin'})


In [37]:
print(args_test('test1', 'test2', 'test3', id='admin', point=7))

<args_test> -> (name: test1) (contents: ('test2', 'test3')) (point: 7) (attrs: {'id': 'admin'})


In [40]:
print(args_test('test1', 'test2', 'test3', id='admin', point=7, password='1234'))

<args_test> -> (name: test1) (contents: ('test2', 'test3')) (point: 7) (attrs: {'id': 'admin', 'password': '1234'})


## (5) 함수 Signature 
: 함수의 인자에 대한 정보를 표시해줄 수 있는 클래스 형태의 메소드

In [41]:
from inspect import signature 

In [42]:
sg = signature(args_test)

print(sg)  # args_test의 파라미터
print(sg.parameters) # args_test의 파라미터 정보가 orderedDict 타입으로 나옴.

(name, *contents, point=None, **attrs)
OrderedDict([('name', <Parameter "name">), ('contents', <Parameter "*contents">), ('point', <Parameter "point=None">), ('attrs', <Parameter "**attrs">)])


- 인자의 모든 정보 출력해보기

In [46]:
for name, params in sg.parameters.items(): 
    print(name, params.kind, params.default, sep=' || ')

name || POSITIONAL_OR_KEYWORD || <class 'inspect._empty'>
contents || VAR_POSITIONAL || <class 'inspect._empty'>
point || KEYWORD_ONLY || None
attrs || VAR_KEYWORD || <class 'inspect._empty'>


## (6) partial
: **인수를 고정하는 용도**
1. 주로 특정 인수를 고정한 후 콜백함수에 사용 
2. 하나 이상의 인수가 이미 할당된 함수의 새 버전 반환 
3. 함수의 새 객체 타입은 이전함수의 자체를 기술하고 있음 

In [48]:
from functools import partial 
from operator import mul

In [50]:
# mul 함수 사용 예
print(10 * 100)
print(mul(10, 100))

1000
1000


- __cf)  mul 함수는 왜 있나__<br>
: mul과 같은 함수를 인수로 받는 경우가 있기 때문이다. 

- 인수 고정

In [52]:
five = partial(mul, 5) # ---> mul(5) 이 상태로 고정시켜 놓겠다. 

In [53]:
six = partial(five, 6) # ---> mul(5, 6) 이 상태로 고정

- 사용

In [54]:
print(five(100)) # 한 개만 받을 수 있음

500


In [55]:
print(six(100)) # 오류. 아무 것도 받을 수 없음 

TypeError: mul expected 2 arguments, got 3

In [56]:
print(six())

30


In [57]:
print([five(i) for i in range(1, 11)])
list(map(five, range(1, 11)))

[5, 10, 15, 20, 25, 30, 35, 40, 45, 50]


[5, 10, 15, 20, 25, 30, 35, 40, 45, 50]